# Processing action units 

In [2]:
import pandas as pd
import pickle

## Loading the file 

In [16]:
filepath = "/Users/dionnespaltman/Desktop/V3/action_units_temp.pkl"

# Load the dataset
action_units = pd.read_pickle(filepath)

In [3]:
print(action_units.columns)

Index(['Frame', 'Face_id', 'Confidence', 'Success', ' AU01_r', ' AU02_r',
       ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r',
       ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r',
       ' AU25_r', ' AU26_r', ' AU45_r', ' AU01_c', ' AU02_c', ' AU04_c',
       ' AU05_c', ' AU06_c', ' AU07_c', ' AU09_c', ' AU10_c', ' AU12_c',
       ' AU14_c', ' AU15_c', ' AU17_c', ' AU20_c', ' AU23_c', ' AU25_c',
       ' AU26_c', ' AU28_c', ' AU45_c', 'ID', 'Timeframe'],
      dtype='object')


In [4]:
display(action_units.head(5))

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Timeframe
103,104,0.0,0.88,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
104,105,0.0,0.98,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
111,112,0.0,0.98,1.0,0.31,0.59,0.0,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
114,115,0.0,0.98,1.0,0.00,0.12,0.0,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
153,154,0.0,0.98,1.0,1.26,1.67,0.0,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"


The temporary data has 184 unique variables (missing IDs from 147 to 287). 

In [5]:
print(len(action_units['ID'].unique()))

184


In [13]:
# Count occurrences of each value in the "Timeframe" column
timeframe_counts = action_units['Timeframe'].value_counts()

print(timeframe_counts)

[4, 5, 6]       2118735
[3, 4, 5, 6]     285556
[7]              269554
[1]              163222
[3]              134364
[2]               78159
[4]               68762
[4, 5]            40338
[4, 6]            15713
Name: Timeframe, dtype: int64


## Missing values 

In [8]:
nan_counts_au_df = action_units.isna().sum()
print(nan_counts_au_df)

Frame              0
Face_id            0
Confidence         0
Success            0
 AU01_r       243394
 AU02_r       243394
 AU04_r       243394
 AU05_r       243394
 AU06_r       243394
 AU07_r       243394
 AU09_r       243394
 AU10_r       243394
 AU12_r       243394
 AU14_r       243394
 AU15_r       243394
 AU17_r       243394
 AU20_r       243394
 AU23_r       243394
 AU25_r       243394
 AU26_r       243394
 AU45_r       243394
 AU01_c       243394
 AU02_c       243394
 AU04_c       243394
 AU05_c       243394
 AU06_c       243394
 AU07_c       243394
 AU09_c       243394
 AU10_c       243394
 AU12_c       243394
 AU14_c       243394
 AU15_c       243394
 AU17_c       243394
 AU20_c       243394
 AU23_c       243394
 AU25_c       243394
 AU26_c       243394
 AU28_c       243394
 AU45_c       243394
ID                 0
Timeframe          0
dtype: int64


## Dealing with missing values - MICE

Link: https://www.machinelearningplus.com/machine-learning/mice-imputation/?utm_content=cmp-true

In [17]:
# need to enable iterative imputer explicitly since its still experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [18]:
# Define imputer
imputer = IterativeImputer(random_state=100, max_iter=10)

In [19]:
action_units.head(10)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Timeframe
0,104,0.0,0.88,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,105,0.0,0.98,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,112,0.0,0.98,1.0,0.31,0.59,0.0,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,115,0.0,0.98,1.0,0.00,0.12,0.0,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,154,0.0,0.98,1.0,1.26,1.67,0.0,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
5,155,0.0,0.98,1.0,1.50,2.48,0.0,0.0,0.60,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
6,156,0.0,0.88,1.0,0.88,2.77,0.0,0.0,0.59,0.02,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
7,157,0.0,0.98,1.0,0.34,2.22,0.0,0.0,0.73,0.00,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
8,160,0.0,0.98,1.0,0.00,1.30,0.0,0.0,0.19,0.00,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,80,"[4, 5, 6]"
9,161,0.0,0.98,1.0,0.00,1.79,0.0,0.0,0.25,0.00,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,80,"[4, 5, 6]"


In [20]:
columns = list(action_units.columns)
print(columns)

['Frame', 'Face_id', 'Confidence', 'Success', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', ' AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', ' AU28_c', 'AU45_c', 'ID', 'Timeframe']


In [21]:
# Use Numeric Features
action_units_train = action_units.loc[:, ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', ' AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', ' AU28_c', 'AU45_c']]
action_units_train.head(10)

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,0.00,0.00,0.0,0.0,1.34,0.22,0.57,1.56,1.52,0.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.00,0.0,0.0,1.34,0.22,0.57,1.56,1.52,0.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.31,0.59,0.0,0.0,0.31,0.31,0.23,0.21,0.62,0.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.12,0.0,0.0,0.32,0.06,0.10,0.60,0.50,0.29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.26,1.67,0.0,0.0,0.39,0.26,0.00,1.22,0.64,0.18,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
5,1.50,2.48,0.0,0.0,0.60,0.26,0.00,1.89,1.02,0.55,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
6,0.88,2.77,0.0,0.0,0.59,0.02,0.00,1.84,1.22,1.00,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
7,0.34,2.22,0.0,0.0,0.73,0.00,0.00,1.88,1.51,1.44,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
8,0.00,1.30,0.0,0.0,0.19,0.00,0.04,1.18,0.69,0.72,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.00,1.79,0.0,0.0,0.25,0.00,0.12,1.57,0.99,0.88,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Running everything up until the imputer was very fast. Running the imputer took around 9 minutes (on the temp data!). 

In [22]:
# fit on the dataset
imputer.fit(action_units_train)

IterativeImputer(random_state=100)

In [23]:
action_units_imputed = imputer.transform(action_units_train)
action_units_imputed[:10]

array([[0.  , 0.  , 0.  , 0.  , 1.34, 0.22, 0.57, 1.56, 1.52, 0.54, 0.11,
        1.96, 1.08, 1.3 , 0.78, 0.  , 0.65, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.34, 0.22, 0.57, 1.56, 1.52, 0.54, 0.11,
        1.96, 1.08, 1.3 , 0.78, 0.  , 0.65, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.31, 0.59, 0.  , 0.  , 0.31, 0.31, 0.23, 0.21, 0.62, 0.12, 0.  ,
        0.04, 0.49, 0.15, 0.  , 0.27, 0.38, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.12, 0.  , 0.  , 0.32, 0.06, 0.1 , 0.6 , 0.5 , 0.29, 0.2 ,
        0.36, 0.91, 0.75, 0.55, 0.  , 0.36, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [1.26, 1.67, 0.  , 0.

In [24]:
# Replace with imputed values
action_units.loc[:, ['AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', ' AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', ' AU28_c', 'AU45_c']] = action_units_imputed
action_units.head(10)

,Frame,Face_id,Confidence,Success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,ID,Timeframe
0,104,0.0,0.88,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
1,105,0.0,0.98,1.0,0.00,0.00,0.0,0.0,1.34,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
2,112,0.0,0.98,1.0,0.31,0.59,0.0,0.0,0.31,0.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
3,115,0.0,0.98,1.0,0.00,0.12,0.0,0.0,0.32,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80,"[4, 5, 6]"
4,154,0.0,0.98,1.0,1.26,1.67,0.0,0.0,0.39,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
5,155,0.0,0.98,1.0,1.50,2.48,0.0,0.0,0.60,0.26,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
6,156,0.0,0.88,1.0,0.88,2.77,0.0,0.0,0.59,0.02,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
7,157,0.0,0.98,1.0,0.34,2.22,0.0,0.0,0.73,0.00,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,80,"[4, 5, 6]"
8,160,0.0,0.98,1.0,0.00,1.30,0.0,0.0,0.19,0.00,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,80,"[4, 5, 6]"
9,161,0.0,0.98,1.0,0.00,1.79,0.0,0.0,0.25,0.00,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,80,"[4, 5, 6]"


In [25]:
nan_counts_au_df = action_units.isna().sum()
print(nan_counts_au_df)

Frame         0
Face_id       0
Confidence    0
Success       0
AU01_r        0
AU02_r        0
AU04_r        0
AU05_r        0
AU06_r        0
AU07_r        0
AU09_r        0
AU10_r        0
AU12_r        0
AU14_r        0
AU15_r        0
AU17_r        0
AU20_r        0
AU23_r        0
AU25_r        0
AU26_r        0
 AU45_r       0
AU01_c        0
AU02_c        0
AU04_c        0
AU05_c        0
AU06_c        0
AU07_c        0
AU09_c        0
AU10_c        0
AU12_c        0
AU14_c        0
AU15_c        0
AU17_c        0
AU20_c        0
AU23_c        0
AU25_c        0
AU26_c        0
 AU28_c       0
AU45_c        0
ID            0
Timeframe     0
dtype: int64


## Saving the action units file with the imputed values 

In [26]:
# action_units.to_pickle("/Users/dionnespaltman/Desktop/V3/action_units_temp_imputed.pkl")

# action_units.to_csv("/Users/dionnespaltman/Desktop/V3/action_units_temp_imputed.csv", index=False)

## TS Fresh requirements

In [27]:
# Necessary imports
!pip install tsfresh

import pandas as pd
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import ComprehensiveFCParameters


## Appling TS Fresh on action units 

In [32]:
# action_units = action_units.rename(columns={' AU45_r': 'AU45_r'})

In [33]:
columns = list(action_units.columns)
print(columns)

['Frame', 'Face_id', 'Confidence', 'Success', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', ' AU28_c', 'AU45_c', 'ID', 'Timeframe']


In [35]:
au_long_format = pd.melt(action_units, id_vars=['ID', 'Frame'],
                                   var_name='Measurement', value_name='Value',
                                   value_vars=['AU01_r', 'AU02_r',
       'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r',
       'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r',
       'AU25_r', 'AU26_r', 'AU45_r'])

In [36]:
display(au_long_format)

,ID,Frame,Measurement,Value
0,80,104,AU01_r,0.00
1,80,105,AU01_r,0.00
2,80,112,AU01_r,0.31
3,80,115,AU01_r,0.00
4,80,154,AU01_r,1.26
...,...,...,...,...
57730567,25,2196,AU45_r,0.00
57730568,25,2197,AU45_r,0.00
57730569,25,2198,AU45_r,0.00
57730570,25,2199,AU45_r,0.00


In [37]:
nan_counts = au_long_format.isna().sum()
print(nan_counts)

ID             0
Frame          0
Measurement    0
Value          0
dtype: int64


In Judita her article, she used the following six intensity level characteristics: sum, variance, standard deviation, maximum, mean, and mean-root square values (total number of extracted features = 102). 

In [41]:
# Convert 'ID' and 'Frame' columns to integer data type
au_long_format['ID'] = au_long_format['ID'].astype(int)
au_long_format['Frame'] = au_long_format['Frame'].astype(int)

# # Convert 'Measurement' column to string data type
au_long_format['Measurement'] = au_long_format['Measurement'].astype(str)

In [43]:
column_types = au_long_format.dtypes
print(column_types)

ID               int64
Frame            int64
Measurement     object
Value          float64
dtype: object


In [45]:
# Specificeer de gewenste kenmerken
settings = {
    # 'sum': {},
    'sum_values': {},
    'variance': {},
    'standard_deviation': {},
    'maximum': {},
    'minimum': {},
    #'median': {},
    'mean': {},
    'mean_abs_change': {},  #proxy voor gemiddelde van de wortel van de kwadraten als 'mean root square' niet direct beschikbaar is
    #'agg_linear_trend': [{'attr': 'slope', 'f_agg': 'max', 'chunk_len': 5}, {'attr': 'slope', 'f_agg': 'min', 'chunk_len': 5}]  #maximale en minimale helling
}

# Extract features met aangepaste instellingen
extracted_features_au = extract_features(
    au_long_format,
    column_id='ID',  # Identifies the time series
    column_sort='Frame',  # Orders the time points
    column_kind='Measurement',  # Distinguishes between different variables if necessary
    column_value='Value',  # The actual measurements
    default_fc_parameters=settings
)


Feature Extraction: 100%|██████████| 20/20 [00:22<00:00,  1.14s/it]


In [46]:
print(extracted_features_au.head)

<bound method NDFrame.head of      AU01_r__sum_values  AU01_r__variance  AU01_r__standard_deviation  \
5               6142.65          0.233452                    0.483169   
6               6802.40          0.437904                    0.661743   
7               7125.50          0.333478                    0.577475   
8               5542.21          0.222518                    0.471719   
9               5854.51          0.164812                    0.405970   
..                  ...               ...                         ...   
328             1543.09          0.225924                    0.475314   
329             1479.36          0.138269                    0.371846   
330             1066.31          0.078431                    0.280056   
331             1691.75          0.216956                    0.465785   
332             1283.27          0.152346                    0.390315   

     AU01_r__maximum  AU01_r__minimum  AU01_r__mean  AU01_r__mean_abs_change  \
5            

## Saving the extracted features to csv and pkl

In [50]:
extracted_features_au.to_csv('/Users/dionnespaltman/Desktop/V3/action_units_temp_extracted.csv', index=True)
extracted_features_au.to_pickle('/Users/dionnespaltman/Desktop/V3/action_units_temp_extracted.pkl')

## Understanding extracted features 

In [52]:
print(extracted_features_au.describe())

       AU01_r__sum_values  AU01_r__variance  AU01_r__standard_deviation  \
count          184.000000      1.840000e+02                1.840000e+02   
mean          7145.876891      3.950921e-01                5.827582e-01   
std           5258.113440      2.933320e-01                2.361951e-01   
min              1.590000      3.081488e-33                5.551115e-17   
25%           1468.632500      2.108621e-01                4.591160e-01   
50%           7317.655000      3.251451e-01                5.702067e-01   
75%          10545.170000      5.197051e-01                7.208982e-01   
max          22053.430000      1.617627e+00                1.271860e+00   

       AU01_r__maximum  AU01_r__minimum  AU01_r__mean  \
count       184.000000       184.000000    184.000000   
mean          4.187974        -0.041374      0.354239   
std           1.081899         0.441383      0.152848   
min           0.380000        -4.330000      0.056170   
25%           3.642500         0.000000

Running the visualization took around ... minutes. (Haven't run it yet)

In [ ]:
# visualization (for example, using seaborn)
import seaborn as sns
sns.pairplot(extracted_features_au)

## Reducing features to the most important ones 
In this example, y represents the target variable you are trying to predict or classify. The select_features function filters out the irrelevant features, keeping only those with significant predictive power.

In [4]:
VVR_scores = pd.read_csv('/Users/dionnespaltman/Desktop/V3/VVR_scores_final.csv')
extracted_features_au = pd.read_csv('/Users/dionnespaltman/Desktop/V3/action_units_temp_extracted.csv')

In [5]:
print(len(VVR_scores['VVR_group']))
print(len(extracted_features_au))

320
184


In [7]:
extracted_features_au = extracted_features_au.rename(columns={'Unnamed: 0': 'ID'})

In [8]:
display(extracted_features_au.head(5))

,ID,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__minimum,AU01_r__mean,AU01_r__mean_abs_change,AU02_r__sum_values,AU02_r__variance,...,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__minimum,AU45_r__mean,AU45_r__mean_abs_change
0,5,6142.65,0.233452,0.483169,4.59,0.0,0.245529,0.093365,2545.74,0.078076,...,0.0,0.321875,0.099693,9502.95,0.365947,0.604936,3.78,0.0,0.379845,0.128288
1,6,6802.40,0.437904,0.661743,5.00,0.0,0.389845,0.131898,3626.81,0.239063,...,0.0,0.676600,0.204811,5834.73,0.311603,0.558214,3.95,0.0,0.334388,0.157706
2,7,7125.50,0.333478,0.577475,5.00,0.0,0.281240,0.252769,5123.91,0.303784,...,0.0,0.660326,0.469192,15768.86,1.250335,1.118184,5.00,0.0,0.622389,0.636920
3,8,5542.21,0.222518,0.471719,4.22,0.0,0.276654,0.068581,2367.16,0.093930,...,0.0,0.701038,0.209352,7791.52,0.409860,0.640203,3.67,0.0,0.388934,0.117176
4,9,5854.51,0.164812,0.405970,4.64,0.0,0.259382,0.092342,2084.46,0.066052,...,0.0,0.729662,0.146979,11029.72,0.698892,0.835998,4.63,0.0,0.488668,0.162506


I can't really do anything further, because I don't have all the video data. But I'll try to match the IDs of what I do have. 

## ...

## ...